In [3]:
# TERRAIN TRANSFORMER - OPTIMIZED FOR SPEED ON RTX 4070 WITH PREPROCESSED DATA
import os
import json
import gc
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Paths
DATA_DIR = 'block_dataset'  # CSV files
OUT_DIR = 'preprocessed_chunks'  # Output .pt files
ENCODER_PATH = 'label_encoders.pkl'
CHUNK_WIDTH, CHUNK_HEIGHT, CHUNK_DEPTH = 16, 256, 16

# Make sure this agrees with the modeling script
# If we want to keep Java unchanged we gotta make it 256
SUB_CHUNK_HEIGHT = 32 

# Create output directory
os.makedirs(OUT_DIR, exist_ok=True)

# Categorical fields
categorical_fields = [
    'Block_ID',
    'Block_to_Left', 'Block_to_Right',
    'Block_Below', 'Block_Above',
    'Block_in_Front', 'Block_Behind',
    'ChunkBiome', 'Biome'
]

# Collect all unique values for encoders
label_encoders = {field: LabelEncoder() for field in categorical_fields}
all_values = {field: set() for field in categorical_fields}

csv_files = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f.endswith('.csv')]

# Pass 1: Collect unique values
print("Scanning files for label encoders...")
for file in tqdm(csv_files):
    df = pd.read_csv(file)
    for field in categorical_fields:
        all_values[field].update(df[field].dropna().unique())

for field in categorical_fields:
    label_encoders[field].fit(list(all_values[field]))

# Save encoders
with open(ENCODER_PATH, 'wb') as f:
    pickle.dump(label_encoders, f)
print(f"Label encoders saved to {ENCODER_PATH}")

# Pass 2: Generate .pt subchunks
print("Generating .pt sub-chunks...")
file_count = 0
for file in tqdm(csv_files):
    df = pd.read_csv(file)

    # Filter invalid rows early
    df = df[(df['x'] < CHUNK_WIDTH) & (df['y'] < CHUNK_HEIGHT) & (df['z'] < CHUNK_DEPTH)]

    # Normalize light
    df['Light_Level'] = df['Light_Level'] / 15.0

    # Encode categorical features
    for key in categorical_fields:
        df[key] = label_encoders[key].transform(df[key])

    # Iterate over vertical slices
    for y_start in range(0, CHUNK_HEIGHT, SUB_CHUNK_HEIGHT):
        y_end = y_start + SUB_CHUNK_HEIGHT
        sub_df = df[(df['y'] >= y_start) & (df['y'] < y_end)]

        # Initialize tensors
        input_tensor = np.zeros((CHUNK_WIDTH, SUB_CHUNK_HEIGHT, CHUNK_DEPTH, 10), dtype=np.float32)
        output_tensor = np.full((CHUNK_WIDTH, SUB_CHUNK_HEIGHT, CHUNK_DEPTH), -1, dtype=np.int64)

        for row in sub_df.itertuples(index=False):
            x, y, z = int(row.x), int(row.y) - y_start, int(row.z)
            if 0 <= x < CHUNK_WIDTH and 0 <= y < SUB_CHUNK_HEIGHT and 0 <= z < CHUNK_DEPTH:
                features = [
                    row.ChunkBiome, row.Biome,
                    float(row.Is_Surface), float(row.Light_Level),
                    row.Block_to_Left, row.Block_to_Right,
                    row.Block_Below, row.Block_Above,
                    row.Block_in_Front, row.Block_Behind,
                ]
                input_tensor[x, y, z] = features
                output_tensor[x, y, z] = row.Block_ID

        # Save to disk
        input_tensor = torch.tensor(input_tensor).permute(3, 0, 1, 2)  # [C, X, Y, Z]
        output_tensor = torch.tensor(output_tensor)  # [X, Y, Z]

        chunk_name = os.path.basename(file).replace('.csv', f'_Y{y_start}.pt')
        torch.save((input_tensor, output_tensor), os.path.join(OUT_DIR, chunk_name))
        file_count += 1

print(f"✅ Preprocessing complete: {file_count} sub-chunks saved to {OUT_DIR}")

Scanning files for label encoders...


100%|██████████| 230/230 [00:23<00:00,  9.78it/s]


Label encoders saved to label_encoders.pkl
Generating .pt sub-chunks...


100%|██████████| 230/230 [01:47<00:00,  2.13it/s]

✅ Preprocessing complete: 1840 sub-chunks saved to preprocessed_chunks


In [4]:
import os
import torch
import pickle
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# === CONFIGURATION ===
DATA_DIR = 'preprocessed_chunks'  # Use preprocessed .pt files
LABEL_ENCODER_PATH = 'label_encoders.pkl'
CHUNK_WIDTH, CHUNK_HEIGHT, CHUNK_DEPTH = 16, 256, 16
SUB_CHUNK_HEIGHT = 32
BATCH_SIZE = 6
EMBED_DIM = 80
NUM_HEADS = 8
NUM_LAYERS = 6
NUM_EPOCHS = 5
LEARNING_RATE = 1e-4
PATIENCE = 5
USE_AMP = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# === DATASET ===
class PreprocessedChunkDataset(Dataset):
    def __init__(self, files, training=True):
        self.files = files
        self.training = training

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        try:
            x, y = torch.load(self.files[idx], weights_only=True)
            if self.training and torch.rand(1).item() > 0.5:
                if torch.rand(1).item() > 0.5:
                    x = torch.flip(x, [1])
                    y = torch.flip(y, [0])
                if torch.rand(1).item() > 0.5:
                    x = torch.flip(x, [3])
                    y = torch.flip(y, [2])
            return x, y
        except Exception as e:
            print(f"[ERROR] Failed loading {self.files[idx]}: {e}")
            return torch.zeros((10, CHUNK_WIDTH, SUB_CHUNK_HEIGHT, CHUNK_DEPTH)), torch.full((CHUNK_WIDTH, SUB_CHUNK_HEIGHT, CHUNK_DEPTH), -1)

# === POSITIONAL ENCODING ===
class FastPositionalEncoding3D(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv_pos = nn.Conv3d(3, channels, kernel_size=1)

    def forward(self, x):
        B, C, X, Y, Z = x.shape
        pos_x = torch.linspace(0, 1, X, device=x.device)
        pos_y = torch.linspace(0, 1, Y, device=x.device)
        pos_z = torch.linspace(0, 1, Z, device=x.device)
        grid_x, grid_y, grid_z = torch.meshgrid(pos_x, pos_y, pos_z, indexing='ij')
        pos = torch.stack([grid_x, grid_y, grid_z], dim=0).unsqueeze(0).expand(B, -1, -1, -1, -1)
        return x + self.conv_pos(pos)

# === ATTENTION BLOCK ===
class FastAttentionBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.norm = nn.GroupNorm(4, dim)
        self.spatial_mixer = nn.Sequential(
            nn.Conv3d(dim, dim, 3, padding=1, groups=dim), nn.GELU(), nn.Conv3d(dim, dim, 1)
        )
        self.channel_mixer = nn.Sequential(
            nn.Conv3d(dim, dim*2, 1), nn.GELU(), nn.Conv3d(dim*2, dim, 1)
        )

    def forward(self, x):
        x_norm = self.norm(x)
        x = x + self.spatial_mixer(x_norm)
        x = x + self.channel_mixer(self.norm(x))
        return x

# === MODEL ===
class FastTerrainTransformer(nn.Module):
    def __init__(self, in_channels, embed_dim, num_layers, num_classes):
        super().__init__()
        self.embedding = nn.Sequential(
            nn.Conv3d(in_channels, embed_dim, 3, padding=1), nn.GELU()
        )
        self.pos_encoding = FastPositionalEncoding3D(embed_dim)
        self.blocks = nn.ModuleList([FastAttentionBlock(embed_dim) for _ in range(num_layers)])
        self.output = nn.Sequential(
            nn.GroupNorm(4, embed_dim), nn.Conv3d(embed_dim, num_classes, 1)
        )

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        for block in self.blocks:
            x = block(x)
        return self.output(x)

# === LOAD LABEL ENCODERS ===
with open(LABEL_ENCODER_PATH, 'rb') as f:
    label_encoders = pickle.load(f)

num_classes = len(label_encoders['Block_ID'].classes_)

# === LOAD DATA ===
all_files = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f.endswith('.pt')]
val_split = 0.1
val_count = int(len(all_files) * val_split)
train_files = all_files[val_count:]
val_files = all_files[:val_count]

train_ds = PreprocessedChunkDataset(train_files, training=True)
val_ds = PreprocessedChunkDataset(val_files, training=False)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

# === TRAIN ===
model = FastTerrainTransformer(10, EMBED_DIM, NUM_LAYERS, num_classes).to(device)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index=-1)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
scaler = torch.amp.GradScaler('cuda' if USE_AMP and device.type == 'cuda' else 'cpu')

best_val_loss = float('inf')
patience_counter = 0

for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    for batch_X, batch_y in tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]"):
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        with torch.amp.autocast(device_type='cuda', enabled=USE_AMP):
            logits = model(batch_X)
            loss = criterion(logits.view(-1, num_classes), batch_y.view(-1))
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in tqdm(val_loader, desc=f"Epoch {epoch+1} [Valid]"):
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            logits = model(batch_X)
            loss = criterion(logits.view(-1, num_classes), batch_y.view(-1))
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    scheduler.step(avg_val_loss)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'terrain_transformer_2.pth')
        print("Saved best model")
    else:
        patience_counter += 1
        print(f"No improvement. Patience: {patience_counter}/{PATIENCE}")
        if patience_counter >= PATIENCE:
            print("Early stopping!")
            break

# Export ONNX
model.eval().to('cpu')
dummy = torch.randn(1, 10, CHUNK_WIDTH, SUB_CHUNK_HEIGHT, CHUNK_DEPTH)
torch.onnx.export(model, dummy, 'terrain_transformer_model.onnx', input_names=['input'], output_names=['output'], dynamic_axes={'input': {0: 'batch'}, 'output': {0: 'batch'}}, verbose=False)
print("Exported to ONNX")

Using device: cuda


Epoch 1 [Valid]: 100%|██████████| 31/31 [00:01<00:00, 18.14it/s]


Epoch 1 | Train Loss: 5.0119 | Val Loss: 4.7796
Saved best model


Epoch 2 [Valid]: 100%|██████████| 31/31 [00:00<00:00, 87.73it/s]


Epoch 2 | Train Loss: 4.4499 | Val Loss: 3.7282
Saved best model


...

Epoch 3 | Train Loss: 3.4710 | Val Loss: 3.0644
Saved best model


Epoch 4 [Valid]: 100%|██████████| 31/31 [00:00<00:00, 87.34it/s]


Epoch 4 | Train Loss: 2.9625 | Val Loss: 2.6852
Saved best model


Epoch 5 [Valid]: 100%|██████████| 31/31 [00:00<00:00, 87.26it/s]


Epoch 5 | Train Loss: 2.6657 | Val Loss: 2.5451
Saved best model
Exported to ONNX


In [5]:
# === EXPORT BLOCK AND BIOME MAPPINGS ===
block_id_mapping = {
    int(i): label_encoders['Block_ID'].classes_[i]
    for i in range(len(label_encoders['Block_ID'].classes_))
}
with open('block_id_mapping.json', 'w') as f:
    json.dump(block_id_mapping, f, indent=2)
print("Saved block_id_mapping.json")

biome_mapping = {
    label_encoders['ChunkBiome'].classes_[i]: int(i)
    for i in range(len(label_encoders['ChunkBiome'].classes_))
}
with open('biome_id_mapping.json', 'w') as f:
    json.dump(biome_mapping, f, indent=2)
print("Saved biome_id_mapping.json")

Saved block_id_mapping.json
Saved biome_id_mapping.json
